# Portfolio Management

A portfolio is a collection of financial investments. Let's see how we construct, analyse and optimise portfolios with Python. Like with every notebook, this one starts with a list of imports.

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf

plt.style.use("ggplot")

## Portfolio Formation

Today we'll form what is known as a 1/N portfolio, with 5 stocks in it. The 1/N refers to the total investment being divided equally among the N stocks that comprise the portfolio - which is 5 in our case today.

Depending on what kind of data we have, portfolio formation can involve different steps.

### Exercise: Pull a Portfolio (Yahoo! Finance)

In this exercise, you'll pull Yahoo Finance data into panel data using yfinance.

Here is an example of how we could join together data from multiple CSV files. Can you adapt this code to instead join together data pulled directly from Yahoo Finance?

```python
tickers = ["TSLA", "MSFT", "AMZN", "META", "NVDA"]
groups = []

for ticker in tickers:
    data = pd.read_csv(ticker + "_2020.csv")
    df.columns = df.columns.droplevel(1)
    data.insert(0, "Ticker", ticker)
    groups.append(data)

multi_df = pd.concat(data)
multi_df
```

Try it out in the code cell below:

In [21]:
tickers = ["TSLA", "MSFT", "AMZN", "META", "NVDA"]
groups = []

for ticker in tickers:
    data = yf.download(ticker, start="2020-01-01", end="2020-12-31", auto_adjust=False)
    data.columns = data.columns.droplevel(1)
    data.insert(0, "Ticker", ticker)
    groups.append(data)

multi_df = pd.concat(groups)
multi_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2020-01-02,TSLA,28.684000,28.684000,28.713333,28.114000,28.299999,142981500
2020-01-03,TSLA,29.534000,29.534000,30.266666,29.128000,29.366667,266677500
2020-01-06,TSLA,30.102667,30.102667,30.104000,29.333332,29.364668,151995000
2020-01-07,TSLA,31.270666,31.270666,31.441999,30.224001,30.760000,268231500
2020-01-08,TSLA,32.809334,32.809334,33.232666,31.215334,31.580000,467164500
...,...,...,...,...,...,...,...
2020-12-23,NVDA,12.974220,13.009250,13.275000,13.005000,13.256000,179144000
2020-12-24,NVDA,12.958764,12.993750,13.132500,12.938750,13.037250,97884000
2020-12-28,NVDA,12.865266,12.900000,13.062500,12.763250,13.062500,212564000


#### Multi-Stock Time Series Data (CRSP)

Maybe instead of Yahoo! Finance, you took advantage of the [College WRDS access](https://www.imperial.ac.uk/admin-services/library/subject-support/business/crsp/), or obtained a CSV with your data in some other way.

Let's work with our panel data as before.

In [22]:
df = pd.read_csv("data/top_six_2020_2025.csv", dayfirst=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/top_six_2020_2025.csv'

In [ ]:
# Pivot the DataFrame to wide format with stocks as columns and dates as index
df_pivot = df.pivot(index='DlyCalDt', columns='Ticker', values='DlyClose')
df_pivot

portfolio_stocks = df_pivot.columns[:5] # Just grab the first 5
df_pivot = df_pivot[portfolio_stocks]

weights = np.ones(5) / 5

## Calculating Portfolio Daily Returns

First, let's determine the returns of our portfolio. We'll need to start with the daily returns of each stock, and then use `np.dot()` to mutliply each column of returns against its weight in the portfolio.

In [ ]:
# Calculate the simple return of each stock in the portfolio
returns = df_pivot.pct_change()
returns = returns.dropna()

# Compute the portfolio returns
portfolio_returns = np.dot(returns, weights)

# While we're at it check the daily volatility (standard deviation of portfolio returns)
daily_volatility = portfolio_returns.std()

## Annualising Returns and Volatility

To provide a broader picture of our portfolio's performance, let's calculate the annualised portfolio return and volatility. These are important metrics for evaluating and discussing portfolios. They are also used frequently in more advanced financial analyses.

In [ ]:
# Define the number of trading days in a year
trading_days = 252

# Annualize the portfolio return
annualised_return = portfolio_returns.mean() * trading_days

# Annualize the portfolio volatility
annualised_volatility = daily_volatility * np.sqrt(trading_days)

## Sharpe Ratio

The Sharpe Ratio is a measure that helps investors understand the risk-adjusted return of an investment. A high Sharpe ratio indicates that the portfolio's returns are higher for each unit of risk taken on. In contrast, a lower Sharpe Ratio indicates a less favorable risk-reward trade-off, with the potential for lower returns relative to the amount of risk being assumed.

The Sharpe Ratio is calculated using annualised portfolio returns, portfolio volatility (as the measure of risk), and the *risk-free rate*. The risk-free rate is often derived from the yield of a theoretically risk-free investment, typically a government bond. The Sharpe Ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk.

### Exercise: Looking Sharpe

Calculate the Sharpe Ratio for our portfolio by applying the formula below.

$$ \text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p} $$

- $R_p$ is our annualised portfolio return
- $R_f$ is the risk-free rate
- $\sigma_p$ is our annualised volatility


In [ ]:
## YOUR CODE GOES HERE